In [25]:
import pandas as pd
import random as rd
import math as m

In [34]:
def split(D):
    used = []
    test = []
    train = []
    while len(test) < len(D) * 0.2:
        i = rd.randint(0, len(D) - 1)
        if i not in used:
            used.append(i)
            test.append(D.iloc[i].values.tolist())
    for i in range(len(D)):
        if i not in used:
            train.append(D.iloc[i].values.tolist())
    return (train, test)

def dist_euc(p1, p2):
    dist = 0
    if len(p1) != len(p2):
        return 0
    else:
        for i in range(len(p1)):
            dist += (p1[i] - p2[i]) ** 2
        return m.sqrt(dist)

def dist_man(p1, p2):
    dist = 0
    if len(p1) != len(p2):
        return 0
    else:
        for i in range(len(p1)):
            dist += p1[i] + p2[i]
        return dist


def dist_cos(p1, p2):
    dist = 0
    if len(p1) != len(p2):
        return 0
    else:
        l = len(p1)
        dot = sum(i[0] * i[1] for i in zip(p1, p2))
        return dot / (dist_euc([0] * l, p1) * dist_euc([0] * l, p2))

train, test = split(pd.read_csv('iris.csv'))

10


Problem 6\
Consider the following algorithm. Run it on Iris.tab and Mnist datasets and report your results.\
    1) Divide your data into random train and test sets\
    2) Training: for a specific label, find the average of each feature. (In case of mnist, this is pixel wise).  give your averages. In case of mint, give this as 10 images, one for each label\
    3) For each data point in the training set, find the closest from the average templates. Use Manhattan, Euclidean and Cosine distances (note that higher cosine value means lower distance). Give the point that label. \
    4) Do the same for the test set\
    5) Give the confusion matrix for training and test sets.\

In [39]:
# Iris Algorithm
def P6(train, test):
    avg = {}
    for row in train:
        if row[4] not in avg:
            avg[row[4]] = [row[0:4], 1]
        else:
            avg[row[4]][1] += 1
            for i in range(len(row) - 1):
                avg[row[4]][0][i] = avg[row[4]][0][i] + (row[i] - avg[row[4]][0][i]) / avg[row[4]][1]

    train_vals = []
    test_vals = []
    for val in avg.values():
        train_vals.append(find_min_dist(val[0], train))
        test_vals.append(find_min_dist(val[0], test))

    print(train_vals)
    print(test_vals)

def find_min_dist(avg, data):
    dist = [m.inf, m.inf, m.inf]
    keys = [[], [], []]
    for i in data:
        d = dist_euc(avg, i[:-1])
        if d < dist[0]:
            dist[0] = d
            keys[0] = i[:-1]

        d = dist_man(avg, i[:-1])
        if d < dist[1]:
            dist[1] = d
            keys[1] = i[:-1]

        d = dist_cos(avg, i[:-1])
        if d < dist[2]:
            dist[2] = d
            keys[2] = i[:-1]

    return keys


P6(train, test)


[[[5.0, 3.4, 1.5, 0.2], [4.5, 2.3, 1.3, 0.3], [7.7, 2.6, 6.9, 2.3]], [[5.7, 2.8, 4.1, 1.3], [4.5, 2.3, 1.3, 0.3], [4.6, 3.6, 1.0, 0.2]], [[6.5, 3.0, 5.5, 1.8], [4.5, 2.3, 1.3, 0.3], [4.6, 3.6, 1.0, 0.2]]]
[[[5.2, 3.4, 1.4, 0.2], [4.3, 3.0, 1.1, 0.1], [5.8, 2.8, 5.1, 2.4]], [[5.7, 2.9, 4.2, 1.3], [4.3, 3.0, 1.1, 0.1], [5.5, 4.2, 1.4, 0.2]], [[6.4, 2.8, 5.6, 2.1], [4.3, 3.0, 1.1, 0.1], [5.5, 4.2, 1.4, 0.2]]]


Problem 7
Consider the following algorithm. Run it on Iris.tab and Mnist datasets and report your results.
    1) Divide your data into random train and test sets
    2) Training: k-means clustering for k=1 to k=15 (i.e., run your algorithm 15 times). DO NOT USE LABLES. Give the centroids. In case of mint, give this as k images, one for each label. (you may use a library function)
    3) Label a cluster using the majority label found in the cluster. 
    4) For each data point in the test set, find the closest from the centroid Use Manhattan, Euclidean and Cosine distances (note that higher cosine value means lower distance). Give the point that label. 
    5) Give the confusion matrix for training and test sets.
    6) Form the elbow map - using %correct as a measure. (Test set accuracy)

Problem 8
Consider the following algorithm. Run it on Iris.tab and Mnist datasets and report your results.
    1) Divide your data into random train and test sets
    2) Using kNN with Manhattan, Euclidean and cosine distances classify the training and test sets. Do this for k=1, k=5, k=10. (you may use a library function)
    3) Give the confusion matrix for training and test sets.
    4) Form the elbow map - using %correct as a measure. (Test set accuracy)